In [1]:
HOME = '/your/HOME/directory'

# 1 Prepare


## 1.1 Prepare 50 papers
1. Download arXiv dataset JSON file from <https://www.kaggle.com/datasets/Cornell-University/arxiv?resource=download>, move to HOME directory.
2. Run the code, get ARXIV_50_JSON
3. Download the 50 papers in ARXIV_50_JSON
4. Manually extracted bandgap from the 50 papers, save results to COMPARISON_XLSX (see example in comparison.xlsx).

In [6]:
import os
import json
from datetime import datetime
from collections import defaultdict
import pandas as pd
from glob import glob

KAGGLE_JSON = os.path.join(HOME, "arxiv-metadata-oai-snapshot.json")
ARXIV_DIR = os.path.join(HOME, "arXiv_mtrl-sci")
os.makedirs(ARXIV_DIR, exist_ok=True)
ARXIV_CSV = os.path.join(HOME, "arXiv_mtrl-sci.csv")
ARXIV_50_JSON = os.path.join(HOME, "arXiv_mtrl-sci_50.json")

# 定义筛选时间范围
START_DATE = datetime(2000, 1, 1)
END_DATE = datetime(2024, 10, 31)

In [7]:
"""filter"""
# 定义一个字典用于存储按年月分组的结果
grouped_data = defaultdict(list)

# 逐行读取 JSON 文件
with open(KAGGLE_JSON, 'r', encoding='utf-8') as f:
    for line in f:
        # 加载一行 JSON 数据
        entry = json.loads(line.strip())
        
        # 筛选条件：categories 包含 'mtrl-sci'
        if 'mtrl-sci' in entry.get('categories', ''):
            # 提取所需字段
            item = {
                'id': entry['id'],
                'doi': entry.get('doi', None),  # 如果没有 DOI，返回 None
                'categories': entry['categories']
            }
            # 提取第一个版本的创建日期并格式化
            first_version = entry['versions'][0]['created']
            date_v1 = datetime.strptime(first_version, '%a, %d %b %Y %H:%M:%S %Z')
            item['date-v1'] = date_v1.strftime('%Y-%m-%d')
            
            # 检查日期是否在指定范围内
            if START_DATE <= date_v1 <= END_DATE:
                # 按年月分组
                year_month = date_v1.strftime('%Y-%m')
                grouped_data[year_month].append(item)

# 将分组后的数据写入 JSON 文件
for year_month, items in grouped_data.items():
    # 获取条目数量
    count = len(items)
    # 在文件名中添加条目数量
    output_file = os.path.join(ARXIV_DIR, f"{year_month}({count}).json")
    with open(output_file, 'w', encoding='utf-8') as out_f:
        json.dump(items, out_f, indent=4, ensure_ascii=False)

In [ ]:
"""randonly choose 50 papers"""
# 1. 读取json文件
json_files = sorted(glob(os.path.join(ARXIV_DIR, '*.json')))

data = []
for json_file in json_files:
    with open(json_file, 'r') as f:
        content = json.load(f)
        for paper in content:
            # 提取年份
            year = paper['date-v1'][:4] if paper['date-v1'] else None
            data.append({
            'year': year,
            'id': paper['id'],
            'doi': paper['doi']
        })

# 2. 创建DataFrame
df = pd.DataFrame(data)

# 3. 统计独立的id和doi数量
unique_ids = df['id'].nunique()
unique_dois = df['doi'].dropna().nunique()  # 排除None值后统计

print(f"独立的文章ID数量: {unique_ids}")
print(f"独立的DOI数量: {unique_dois}")

# 4. 保存为CSV文件
df.to_csv(ARXIV_CSV, index=False)
print(f"\n数据已保存到: {ARXIV_CSV}")

# 5. 随机选取50篇文章
# 只保留同时包含id和doi的文章
df_complete = df.dropna(subset=['id', 'doi'])
df_sample = df_complete.sample(n=50, random_state=42)
df_sample.to_json(ARXIV_50_JSON, orient='records', indent=2)
print(f"已将50篇文章样本保存到: {ARXIV_50_JSON}")

# 6. 统计年份分布
year_distribution = df_sample['year'].value_counts().sort_index()
print("\n年份分布情况:")
print(year_distribution)

# 可视化年份分布
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
year_distribution.plot(kind='bar')
plt.title('Sample Articles Year Distribution')
plt.xlabel('Year')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 1.2 Sparse PDF to TXT

1. Download the 50 papers' PDF files to PDF_DIR
2. Run the code to get TXT_DIR_2 (TXT files, with papers in sentences)

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
import os
import glob
import spacy

PDF_DIR = os.path.join(HOME, "PDF")
os.makedirs(PDF_DIR, exist_ok=True)
TXT_DIR_1 = os.path.join(HOME, "TXT(fromPDF)")
os.makedirs(TXT_DIR_1, exist_ok=True)
TXT_DIR_2 = os.path.join(HOME, "TXT(fromPDF_processed)")
os.makedirs(TXT_DIR_2, exist_ok=True)

"""Sparse"""
# 遍历每个pdf，将每个pdf的text输出到txt文件中，保存在TXT_DIR目录下，文件名与pdf文件名一致
pdf_files = sorted(glob.glob(os.path.join(PDF_DIR, "*.pdf")))
for pdf_file in pdf_files:
    loader = PyMuPDFLoader(pdf_file)
    docs = loader.load()
    text = ""
    for doc in docs:
        text += doc.page_content
    with open(os.path.join(TXT_DIR_1, os.path.basename(pdf_file).replace('.pdf', '.txt')), 'w') as f:
        f.write(text)



"""Process"""
nlp = spacy.load("en_core_web_sm")

# 常量定义
EXCLUDED_ENDINGS = ('Fig.', 'Eq.', 'Figs.', 'et al.')
SENTENCE_ENDINGS = ('.', '!', '?')
MIN_SENTENCE_LENGTH = 20

def should_merge(sentence: str) -> bool:
    """判断是否需要合并下一个句子"""
    if not sentence:
        return False
    # 如果不以标准句子结尾符结束，或者以排除项结尾，则需要合并
    return (not sentence.endswith(SENTENCE_ENDINGS) 
            or any(sentence.endswith(end) for end in EXCLUDED_ENDINGS))

def merge_consecutive(sentences: list[str]) -> list[str]:
    """合并需要连接的连续句子"""
    merged = []
    i, n = 0, len(sentences)
    
    while i < n:
        current = sentences[i].strip()
        j = i + 1
        
        # 连续合并需要连接的句子
        while j < n and should_merge(current):
            current += " " + sentences[j].strip()
            j += 1
        
        merged.append(current)
        i = j  # 移动到下一个未处理的句子
    
    return merged

def merge_short_sentences(sentences: list[str]) -> list[str]:
    """合并过短的句子到前一句"""
    merged = []
    for sentence in sentences:
        if not sentence:
            continue
        
        # 如果当前句子过短且结果列表不为空，则合并到前一句
        if merged and len(sentence) < MIN_SENTENCE_LENGTH:
            merged[-1] += " " + sentence
        else:
            merged.append(sentence)
    
    return merged

def process_txt(input_path: str, output_path: str) -> None:
    # 读取并预处理文本
    with open(input_path, "r", encoding="utf-8") as file:
        raw_text = file.read().replace("\n", " ")
    
    # 初始分句
    doc = nlp(raw_text)
    initial_sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
    
    # 多阶段处理流程
    processed = merge_consecutive(initial_sentences)
    processed = merge_short_sentences(processed)
    processed = merge_consecutive(processed)  # 处理短句合并后的新情况
    
    # 写入结果
    with open(output_path, "w", encoding="utf-8") as file:
        file.write("\n".join(processed))

# 读取OUTPUT1_DIR下的所有txt，使用process_txt依次处理后，保存至OUTPUT2_DIR
for txt_file in glob.glob(os.path.join(TXT_DIR_1, "*.txt")):
    process_txt(txt_file, os.path.join(TXT_DIR_2, os.path.basename(txt_file)))

# 2 Extraction

1. Download projects to PROJECT_DIR
   1. <https://github.com/QingyangDong-qd220/BandgapDatabase1>
   2. <https://github.com/StefanoSanvitoGroup/BERT-PSIE-TC>
2. Download LLMs
   1. <https://huggingface.co/m3rg-iitd/matscibert>
   2. <https://ollama.com/library/nomic-embed-text> (ollama pull nomic-embed-text)
   3. <https://ollama.com/library/bge-m3> (ollama pull bge-m3)
   4. <https://ollama.com/library/llama2> (ollama pull llama2:13b)
   5. <https://huggingface.co/bartowski/Llama-3.1-Nemotron-70B-Instruct-HF-GGUF> (ollama create llama3.1:70b -f Modelfile)
   6. <https://ollama.com/library/qwen2.5> (ollama pull qwen2.5:14b)
3. Use PROMPT below to extract from Kimi and place the output to KIMI_OUT



PROMPT:

You are an expert information extraction algorithm.
Extract all the band gap values in this article and output them in the form of a markdown table, including: Material (name of the material), Value (value with unit), Sentence (the sentence from which this data record comes).
If data is not present in the article, type "None". 
Table only, no need for explanation or any other content.
The output is strictly in the following format.
```markdown
| Material | Value | Sentence |
|----------|-------|---------|
| Material1 | 0.1 eV | ... Eg of Material1 is 0.1 eV ... |
| Material1 | 200 meV | Material1 has a band gap of 200 meV, so ... |
| Material2 | None | Material2 ... |
```

If no band gap values mentioned in the article, the following table is acceptable:
```markdown
| Material | Value | Sentence |
|----------|-------|----------|
| None | None | None |
```



## 1-CDE
docker

**docker 前期准备**

chemdataextractor2 无法导入 ➡️ 把"/usr/local/lib/python3.8/site-packages/chemdataextractor"改为"chemdataextractor2"

In [ ]:
!pip install playsound openpyxl

In [ ]:
"""extract"""
import os
import joblib
from pprint import pprint
from tqdm import tqdm
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from chemdataextractor2.relex import Snowball
from chemdataextractor2.model.units.energy import EnergyModel
from chemdataextractor2.model import BaseModel, StringType, ListType, ModelType, Compound
from chemdataextractor2.parse import R, I, W, Optional, merge, join, AutoSentenceParser
from chemdataextractor2.doc import Sentence, Document

class BandGap(EnergyModel):
    specifier_expression = (
        (I("band") + R("gaps?")) | I("bandgap") | I("band-gap") | I("Eg")
    ).add_action(join)
    specifier = StringType(
        parse_expression=specifier_expression, required=True, updatable=True
    )
    compound = ModelType(
        Compound, required=True, contextual=True, binding=True, updatable=False
    )
    parsers = [AutoSentenceParser()]

def run(file_path, article):
    """
    use snowball to serialize an article
    """

    results = []
    # load a paper
    try:
        d = Document.from_file(file_path)
    except:
        print("unable to read document")
        return

    publisher = "arXiv"

    # process a paper
    for p in d.paragraphs:
        for s in p.sentences:
            if s.end - s.start > 300:
                continue

            results_snow = []
            results_auto = []
            snow_85 = False

            """nwk: 第一种提取，使用 AutoSentenceParser，输出在 results_auto"""
            # auto
            BandGap.parsers = [AutoSentenceParser()]
            s.models = [BandGap]  # nwk: 提取
            auto = s.records.serialize()  # nwk: 解析
            for i in auto:
                if "BandGap" in i.keys():  # nwk: 存在带隙相关数据
                    if (
                        "raw_value" in i["BandGap"].keys()
                        and "compound" in i["BandGap"].keys()
                    ):  # nwk: 存在带隙值、材料名
                        if "names" in i["BandGap"]["compound"]["Compound"].keys():
                            i["BandGap"][
                                "text"
                            ] = s.text  # nwk: 在 text 条目储存原始的输出？
                            i['BandGap']['doi'] = article.replace('_', '/').replace('.html', '').replace('.xml', '').replace('.txt', '')  # nwk: 提取文件名中的 doi（所以文件名需要是 doi）
                            results_auto.append(i)

            """nwk: 第二种提取，使用 snowball，输出在 results_snow"""
            # snow
            snowball.minimum_cluster_similarity_score = 0.85
            BandGap.parsers = [snowball]
            s.models = [BandGap]  # nwk: 提取
            snow = s.records.serialize()  # nwk: 解析
            for i in snow:
                if "BandGap" in i.keys():  # nwk: 这里只要求有 bandgap 相关的条目
                    snow_85 = True
                    i["BandGap"]["text"] = s.text
                    i['BandGap']['doi'] = article.replace('_', '/').replace('.html', '').replace('.xml', '').replace('.txt', '')
                    results_snow.append(i)

            if snow_85 == False:  # nwk: 如果“minimum_cluster_similarity_score”参数为 0.85 提取不到数据时，换用 0.65
                snowball.minimum_cluster_similarity_score = 0.65
                BandGap.parsers = [snowball]
                s.models = [BandGap]
                snow = s.records.serialize()
                for i in snow:
                    if "BandGap" in i.keys():
                        i["BandGap"]["text"] = s.text
                        i['BandGap']['doi'] = article.replace('_', '/').replace('.html', '').replace('.xml', '').replace('.txt', '')
                        results_snow.append(i)

            """nwk: 提取完成，将 results_snow 整合进 results_auto"""
            # combine results from Snowball to AutoSentenceParser
            for i in results_auto:
                i["BandGap"]["AutoSentenceParser"] = 1
                i["BandGap"]["Snowball"] = 0
                for j in range(len(results_snow)):
                    if i['BandGap']['compound']['Compound']['names'] == results_snow[j]['BandGap']['compound']['Compound']['names']:  # nwk: 整合相同的材料
                        i["BandGap"] = results_snow[j]["BandGap"]  # nwk: 合并
                        i["BandGap"]["Snowball"] = 1
                        i["BandGap"]["AutoSentenceParser"] = 1
                        results_snow[j]["BandGap"]["match"] = 1  # nwk: 标记
                        continue

            """nwk: 通过上一步的 match 标记，将 auto 没有提取到的数据 results_snow 添加进 results_auto"""
            # Snowball only results
            for x in results_snow:
                if "match" not in x["BandGap"].keys():
                    x["BandGap"]["Snowball"] = 1
                    x["BandGap"]["AutoSentenceParser"] = 0
                    results_auto.append(x)

            """nwk: 最后给条目添加出版商信息"""
            if results_auto:
                for i in results_auto:
                    i["BandGap"]["publisher"] = publisher
                    results.append(i)

    return results

dtime = datetime.now().strftime("%m%d-%H%M")
HOME_DOCKER = "/home/chemdataextractor2/output/HOME_0325"
PROJECT_DIR = os.path.join(HOME_DOCKER, "project")
TXT_DIR = os.path.join(HOME_DOCKER, "TXT(fromPDF_processed)")
OUTPUT_DIR = os.path.join(HOME_DOCKER, "output", "1-ChemDataExtractor")
os.makedirs(OUTPUT_DIR, exist_ok=True)
TEMP_SAVE = os.path.join(OUTPUT_DIR, "records_general.joblib")
RUNTIME = os.path.join(OUTPUT_DIR, f'runtime_{dtime}.txt')

SNOWBALL_PATH = os.path.join(PROJECT_DIR, "BandgapDatabase1-main", "Snowball_model", "general.pkl")

"""load snowball model"""
snowball = Snowball.load(SNOWBALL_PATH)
snowball.minimum_relation_confidence = 0.001
snowball.max_candidate_combinations = 100
snowball.save_file_name = "general"  # model_name
snowball.set_learning_rate(0.0)

# load already found records
try:
    records = joblib.load(TEMP_SAVE)
    print("load existing records")
except:
    records = []
    print("no records found")

start_time = datetime.now()

for file_name in tqdm(os.listdir(TXT_DIR), desc="Processing files"):
    if file_name.endswith('.txt'):
        file_path = os.path.join(TXT_DIR, file_name)
        temp = run(file_path, file_name)
        # save records
        if temp:
            pprint(temp)
            for record in temp:
                records.append(record)
            joblib.dump(records, TEMP_SAVE)

end_time = datetime.now()
run_time = end_time - start_time

with open(RUNTIME, 'w') as f:
    f.write(f'Total runtime: {run_time}')

# took 3:37:29(217m 30.1s)
# took (233m 9.0s)

In [ ]:
"""postprocessing(kernel: lc)"""
import os
import pandas as pd
import joblib
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from my_post import clean_and_normalize, compare

def post_cde(temp_save, xlsx_path, comparison_xlsx, code):
    """ChemDataExtractor提取的原始数据转换为标准格式
    
    Args:
        temp_save: CDE提取的原始数据文件路径
        xlsx_path: 输出的Excel文件路径: "FINAL_{CODE}_{dtime}.xlsx"
    """
    records = joblib.load(temp_save)
    # 提取需要的字段
    columns = ["Publisher", "DOI", "Name", "Raw_value", "Raw_unit", "Value", "Unit", 
              "specifier", "Text", "Snowball", "AutoSentenceParser"]
    flat_data = []
    for item in records:
        bandgap = item["BandGap"]
        flat_item = {
            "Publisher": bandgap["publisher"],
            "DOI": bandgap["doi"],
            "Name": bandgap["compound"]["Compound"]["names"][0],
            "Raw_value": bandgap["raw_value"],
            "Raw_unit": bandgap["raw_units"], 
            "Value": bandgap["value"],
            "Unit": bandgap["units"],
            "specifier": bandgap["specifier"],
            "Text": bandgap["text"],
            "Snowball": bandgap["Snowball"],
            "AutoSentenceParser": bandgap["AutoSentenceParser"]
        }
        flat_data.append(flat_item)
    # 保存原始数据
    df = pd.DataFrame(flat_data, columns=columns)
    with pd.ExcelWriter(xlsx_path, mode="w", engine="openpyxl") as writer:
        df.to_excel(writer, sheet_name="0-raw", index=False)
    # 保存简化数据
    simplified_df = pd.DataFrame({
        'doi': df['DOI'],
        'material': df['Name'],
        'value': df['Raw_value'], 
        'unit': df['Raw_unit']
    })
    with pd.ExcelWriter(xlsx_path, mode="a", if_sheet_exists="replace", engine="openpyxl") as writer:
        simplified_df.to_excel(writer, sheet_name="1-raw", index=False)
    with pd.ExcelWriter(comparison_xlsx, mode="a", if_sheet_exists="replace", engine="openpyxl") as writer:
        simplified_df.to_excel(writer, sheet_name=f"{code}_raw", index=False)

def postprocess_cde(temp_save, xlsx_path, comparison_xlsx, code):
    """CDE提取流程的主函数
    
    Args:
        temp_save: CDE提取的原始数据文件路径（joblib文件）
        xlsx_path: 结果Excel - "FINAL_{CODE}_{dtime}.xlsx"
        comparison_xlsx: 标准答案文件路径
        code: 提取方法代码
    """
    post_cde(temp_save, xlsx_path, comparison_xlsx, code)
    clean_and_normalize(xlsx_path, comparison_xlsx, code, sheet_name="1-raw")
    compare(xlsx_path, comparison_xlsx, code)

"""此处的路径是docker之外的"""
dtime = datetime.now().strftime("%m%d-%H%M")
OUTPUT_DIR = os.path.join(HOME, "output", "1-ChemDataExtractor")
os.makedirs(OUTPUT_DIR, exist_ok=True)
TEMP_SAVE = os.path.join(OUTPUT_DIR, "records_general.joblib")
COMPARISON_XLSX = os.path.join(HOME, "comparison.xlsx")
CODE = "CDE"
XLSX_PATH = os.path.join(HOME, f"1_{CODE}_{dtime}.xlsx")

postprocess_cde(TEMP_SAVE, XLSX_PATH, COMPARISON_XLSX, CODE)


## 2-PSIE
kernel: lc

In [ ]:
"""extract"""
import os
from datetime import datetime
import time
dtime = datetime.now().strftime("%m%d-%H%M")
import json
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, random_split
from transformers import BertTokenizerFast
from seqeval.metrics import classification_report  # 用于评估序列标注任务的性能
import nltk  # 用于自然语言处理任务
import re
from pymatgen.core import Composition  # 用于处理化学式和晶体结构
from datasets import load_dataset

DATA_DIR = os.path.join(HOME, "TXT(fromPDF_processed)")
# model
PROJECT_DIR = os.path.join(HOME, "project")
PSIE_DIR = os.path.join(PROJECT_DIR, "BERT-PSIE-TC-main", "workflow")
import sys
sys.path.insert(1, PSIE_DIR)
import psie
MODEL_DIR = os.path.join(PSIE_DIR, "models", "Gap")
CLASSIFIER_PATH = os.path.join(MODEL_DIR, "classifier.pt")
NER_PATH = os.path.join(MODEL_DIR, "ner")
RELATION_PATH = os.path.join(MODEL_DIR, "relation")
BERT_VERSION = "/Volumes/External/model_cache/huggingface/models--m3rg-iitd--matscibert/snapshots/24a4e4318dda9bc18bff5e6a45debdcb3e1780e3"
MAX_LEN = 256  # 文本序列的最大长度为256

# output
OUTPUT_DIR = os.path.join(HOME, "output", "2-BERT-PSIE")
os.makedirs(OUTPUT_DIR, exist_ok=True)
SENTENCES_JSON = os.path.join(OUTPUT_DIR, "sentences.json")
OUTPUT_JSON_1 = os.path.join(OUTPUT_DIR, "1-relevant_sentences.json")
OUTPUT_JSON_2_M = os.path.join(OUTPUT_DIR, "2-test_extraction_multiple_mentions.json")
OUTPUT_CSV_2_S = os.path.join(OUTPUT_DIR, "2-test_extraction_single_mentions.csv")
OUTPUT_CSV_3 = os.path.join(OUTPUT_DIR, "3-relations_extraction.csv")
RUNTIME = os.path.join(OUTPUT_DIR, f'runtime_{dtime}.txt')

device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
print(device)

start_time_total = time.time()

"""process sentences to json"""
# 定义函数来处理文本文件并生成句子列表
def process_txt_files(data_dir):
    sentences_list = []
    for filename in os.listdir(data_dir):
        if filename.endswith('.txt'):
            doi = filename[:-4]  # 移除 .txt 后缀
            with open(os.path.join(data_dir, filename), 'r', encoding='utf-8') as file:
                for line in file:
                    sentence = line.strip()
                    if sentence:
                        sentences_list.append({"sentence": sentence, "source": doi})
    return sentences_list

# 处理文本文件并生成句子列表
sentences = process_txt_files(DATA_DIR)
# 将句子列表写入 JSON 文件
with open(SENTENCES_JSON, 'w', encoding='utf-8') as json_file:
    json.dump(sentences, json_file, ensure_ascii=False, indent=2)
print(f"已将 {len(sentences)} 个句子写入 {SENTENCES_JSON}")

"""1/3 Classifier"""
start_time_classifier = time.time()

'''tokenize'''
dataset = load_dataset(path="json", data_files=SENTENCES_JSON, split="train")
tokenizer = BertTokenizerFast.from_pretrained(BERT_VERSION)  # 使用MatSciBERT

def encode(paper):  # 使用tokenizer对paper中的句子进行分词、编码和填充处理
  return tokenizer(paper["sentence"], truncation=True, max_length=MAX_LEN, padding="max_length")

dataset = dataset.map(encode, batched=True)  # 以“批处理模式”对数据集的每一个数据应用encode函数

dataset.set_format(type="torch", columns=["source", "sentence", "input_ids", "attention_mask"])  # 设置数据集的格式为PyTorch格式，选择需要的列
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)  # 创建一个数据加载器，用于按批次加载数据集，每批次大小为32，不打乱数据顺序


'''classify'''
model = psie.classifier.BertClassifier()  # 实例化psie模块中的BertClassifier类，创建一个BERT分类模型
# model.load_state_dict(torch.load(MODEL_PATH), device=device)  # 加载预训练的模型参数
state_dict = torch.load(CLASSIFIER_PATH, map_location=device)
state_dict.pop("bert.embeddings.position_ids", None)  # 删除不需要的键
model.load_state_dict(state_dict)
model.to(device)  # 将模型移到GPU上

pred = model.predict(dataset_loader, device)  # 对数据集应用分类模型，得到分类结果

predictions = []
for i in range(len(pred)):
    predictions.append(np.argmax(pred[i].cpu().numpy()))  # 概率最大的类别索引

# 将过滤得到的“相关句子”存为json文件（包含句子和doi）
filtered_sentences = {"sentence": [], "source":[]}

for i in range(len(predictions)):
  if predictions[i] == 1:
    filtered_sentences["sentence"].append((dataset[i]["sentence"]))
    filtered_sentences["source"].append((dataset[i]["source"]))

os.makedirs(os.path.dirname(OUTPUT_JSON_1), exist_ok=True)
with open(OUTPUT_JSON_1, "w") as f:
    json.dump(filtered_sentences, f)

classifier_time = time.time() - start_time_classifier

"""2/3 NER"""
start_time_ner = time.time()

# 设置实体标签的映射关系
id_to_BOI = {
    1: "B-CHEM",  # Chemical entity
    0: "O",  # No entity
    2: "B-BANDGAP"
}
with open(OUTPUT_JSON_1, "r") as f:
    data = json.load(f)  # 加载JSON数据
tokenizer = BertTokenizerFast.from_pretrained(NER_PATH)  # 使用预训练的NER模型

sentences = psie.NerUnlabeledDataset(data["sentence"], tokenizer, max_len=MAX_LEN)  # 传入语料库中的句子数据、分词器和最大长度参数
sources = data["source"]  # 获取语料库中的数据源信息
sentences_params = {'batch_size': 10,
                    'shuffle': False,
                    'num_workers': 0
}
sentences_loader = DataLoader(sentences, **sentences_params)  # 创建一个数据加载器，用于加载句子数据

model = psie.BertForNer.from_pretrained(NER_PATH, num_labels=3)  # 实例化预训练的 NER 模型，标签数量为3
model.to(device)
# NER predictions
predictions = model.predict(sentences_loader, device, id_to_BOI)

# 对每个句子的预测结果进行处理，提取其中的实体标签。首先对句子进行分词和预处理，然后根据预测结果和分词结果提取实体标签，并将结果保存到extr_labels列表中。
extr_labels = []
for n in range(len(predictions)):
    tokens = tokenizer.tokenize(
        "[CLS]" + psie.preprocess_text(sentences[n]["plain"]) + "[SEP]",
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
    )
    extracted = {}
    i = 0
    while i < MAX_LEN:
        if predictions[n][i] != "O" and tokens[i] not in ["[CLS]", "[SEP]", "[PAD]"]:
            entity = predictions[n][i]
            entry = []
            while predictions[n][i] == entity:
                entry.append(tokens[i])
                i += 1
                if i >= MAX_LEN:
                    break
            if entity in extracted.keys():
                extracted[entity].append(" ".join(entry))
            else:
                extracted[entity] = [" ".join(entry)]
        i += 1
    extr_labels.append(extracted)

'''multiple mentioned'''
# Extract the sentences with multiple mentions of Chem and Tc/Gap
# The extracted sentences are saved in a json file and will be processed by the BERT model finetuned for relation classification.
#
relational = []

for i in range(len(extr_labels)):
  n_entries = [len(extr_labels[i][key]) for key in extr_labels[i].keys()]
  if n_entries != []:
    if len(n_entries) == 2:
      if n_entries[0] > 1 and n_entries[1] > 1:
        relational.append(extr_labels[i].copy())

        relational[-1]["sentence"] = sentences[i]["plain"]
        relational[-1]["source"] = sources[i]

print("Relational/Total: ", len(relational), "/", len(predictions))
with open(OUTPUT_JSON_2_M, "w") as f:
    json.dump(relational, f)

'''single mentioned'''
# Extract the sentences with exactly 1 mention of Chem and 1 mention of Tc/Gap
relevant = []

for i in range(len(extr_labels)):
    n_entries = [len(extr_labels[i][key]) for key in extr_labels[i].keys()]
    if n_entries == [1, 1]:
        relevant.append(extr_labels[i])

        relevant[-1]["sentence"] = sentences[i]["plain"]
        relevant[-1]["source"] = sources[i]

print("Relevant/Total: ", len(relevant), "/", len(predictions))

# Cleaning of the sentences with single mentions of CHEM and Tc/Gap at every step the sentences that raise an exception are printed for debugging purpose
# 对上面的单对“化合物-性质”的句子进行清洗（整理），储存到database字典中，并把异常的句子打印出来以供调试
database = {"compound": [], "Gap": [], "sentence": [], "source": []}

for n in range(len(relevant)):
    chem, trgt = None, None

    try:
        chem = (
            relevant[n]["B-CHEM"][0]
            .strip()
            .replace(" ", "")
            .replace("#", "")
            .replace("(", "\(")
            .replace(")", "\)")
            .replace("+", "\+")
            .replace("[UNK]", "")
            .replace(".", "\.")
        )

        chem = re.findall(
            "(?i)[^a-zA-Z0-9]*" + chem + "[^a-zA-Z]",
            relevant[n]["sentence"],
        )[0].strip()

        if chem.endswith(",") or chem.endswith("."):
            chem = chem[0 : len(chem) - 1]
        if chem.startswith(",") or chem.startswith("."):
            chem = chem[1 : len(chem)]

        if chem in psie.ELEMENT_NAMES:
            chem = psie.ELEMENTS[psie.ELEMENT_NAMES.index(chem)]

        trgt = relevant[n][id_to_BOI[2]][0].replace("#", "").strip()
        trgt = (
            trgt.replace("[", "")
            .replace("]", "")
            .replace("{", "")
            .replace("}", "")
            .replace("=", "")
            .replace("[UNK]", "")
        )

        trgt = trgt.replace("ev", "eV")

        if trgt.endswith(",") or trgt.endswith("."):
            trgt = trgt[0 : len(trgt) - 1]
        if trgt.startswith(",") or trgt.startswith("."):
            trgt = trgt[1 : len(trgt)]

        if (chem is not None) and (trgt is not None):
            database["compound"].append(chem)
            database["Gap"].append(trgt)

        database["sentence"].append(relevant[n]["sentence"])
        database["source"].append(relevant[n]["source"])

    except:
        comp = (
            relevant[n]["B-CHEM"][0]
            .replace("#", "")
            .replace(" ", "")
            .replace("(", "\(")
            .replace(")", "\)")
            .replace("+", "\+")
            .replace("[UNK]", "")
        )
        trgt = relevant[n][id_to_BOI[2]][0].replace("#", "").strip()
        print(comp, trgt, relevant[n]["sentence"], "\n\n")  ### Print the cases that raise an exception (for debugging purposes)
# 清洗后的条目数 / 原始句子数
print("Database entries:", len(database["compound"]), "/", len(relevant))

# The chemical entity is converted to a Composition object from pymatgen and its reduced formula is taken
database = pd.DataFrame(database)  # 将清洗后的数据转为DataFrame

valid_i = []

for i, mat in enumerate(database["compound"]):
    try:
        Composition(mat).get_reduced_formula_and_factor()[
            0
        ]  # 使用pymatgen库将化学实体转换为Composition对象，并获取其简化化学式
        valid_i.append(i)
    except:
        print(
            mat, "\t", database["sentence"][i], "\n\n"
        )  # The entries that raise an exception are printed for debugging purpose
# 通过验证的条目数 / 原始数据中的句子总数
print("Database entries:", len(valid_i), "/", len(relevant))
# 将通过验证的条目导出为CSV文件
database.iloc[valid_i].to_csv(OUTPUT_CSV_2_S)

ner_time = time.time() - start_time_ner

"""3/3 Relation"""
start_time_relation = time.time()

# Adding the tokens for the relation extraction step to the BERT models vocabulary so that this tags are not splitted into different subwords.将关系提取步骤的标记添加到 BERT 模型词汇中，这样标记就不会被分割成不同的子词。
tokenizer = BertTokenizerFast.from_pretrained(RELATION_PATH)
new_tokens = ["[E1]", "[/E1]", "[E2]", "[/E2]"]
tokenizer.add_tokens(list(new_tokens))  # 添加关系提取步骤的新标记


# 加载数据
with open(OUTPUT_JSON_2_M, "r") as f:
    data = json.load(f)
print(data[0])


# 给实体（材料和gap）的前后添加标签
data = psie.fromNer(data)

print(data["sentence"][0])
print(data["sentence"][1])


# 如果句子同时包含[E1]和[E2]，则填充ner_dataset
ner_dataset = {"sentence": [], "isrelated": [], "source": []}
for i in range(len(data["sentence"])):
    if ("[E1]" in data["sentence"][i]) and ("[E2]" in data["sentence"][i]):
        ner_dataset["sentence"].append(str(data["sentence"][i]))
        ner_dataset["isrelated"].append(None)
        ner_dataset["source"].append(data["source"][i])
print(len(ner_dataset["sentence"]), "/", len(data["sentence"]))

ner = psie.RelationDataset(
    ner_dataset, tokenizer, max_len=MAX_LEN
)  # 使用psie.RelationDataset将数据转换为模型可接受的格式
ner_params = {"batch_size": 8, "shuffle": False, "num_workers": 0}
ner_loader = DataLoader(ner, **ner_params)  # 按要求加载数据

model = psie.BertForRelations(
    pretrained=RELATION_PATH, dropout=0.2, use_cls_embedding=True
)
model.bert.resize_token_embeddings(len(tokenizer))  # 调整模型的嵌入层大小以适应新的标记数
model.to(device)

# Predictions on the BERT NER output
pred = model.predict(ner_loader, device)  # NER预测
predictions = []  # 将每个预测结果的最大值索引添加到predictions列表中
for i in range(len(pred)):
    predictions.append(np.argmax(pred[i].cpu().numpy()))

# 存储提取的关系数据
database = {"compound": [], "Gap": [], "sentence": [], "source": []}
for i in range(len(predictions)):
    if predictions[i] == 1:
        # 材料
        comp = re.findall(
            re.escape("[E1]") + ".*" + re.escape("[/E1]"), ner_dataset["sentence"][i]
        )
        # 带隙（这里temp是Tc的表示）
        temp = re.findall(
            re.escape("[E2]") + ".*" + re.escape("[/E2]"), ner_dataset["sentence"][i]
        )

        if (len(comp) > 0) and (len(temp) > 0):
            comp = comp[0].replace("[E1]", "").replace("[/E1]", "").replace(" ", "")
            temp = temp[0].replace("[E2]", "").replace("[/E2]", "").replace(" ", "")
            database["compound"].append(comp)
            database["Gap"].append(temp)
            database["sentence"].append(ner_dataset["sentence"][i])
            database["source"].append(ner_dataset["source"][i])

# The chemical entity is converted to a Composition object from pymatgen and its reduced formula is taken 从 pymatgen 将化学实体转换为组成对象，并提取其还原公式
database = pd.DataFrame(database)

valid_i = []
for i, comp in enumerate(database["compound"]):
    try:
        Composition(comp).get_reduced_formula_and_factor()[0]
        valid_i.append(i)
    except:
        print(
            comp, "\t", database["sentence"][i], "\n\n"
        )  # The entries that raise an exception are printed for debugging purpose
print("Database entries:", len(valid_i), "/", len(database["sentence"]))

database.iloc[valid_i].to_csv(OUTPUT_CSV_3)

relation_time = time.time() - start_time_relation
total_time = time.time() - start_time_total

# 记录运行时间
with open(RUNTIME, 'w') as f:
    f.write(f"Classifier time: {classifier_time:.2f} seconds\n")
    f.write(f"NER time: {ner_time:.2f} seconds\n")
    f.write(f"Relation time: {relation_time:.2f} seconds\n")
    f.write(f"Total time: {total_time:.2f} seconds\n")

In [ ]:
"""postprocessing"""
import os
import pandas as pd
import joblib
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from my_post import clean_and_normalize, compare

def post_psie(csv_2s, csv_3, xlsx_path, comparison_xlsx, code):
    """BERT-PSIE提取的原始数据转换为标准格式
    
    Args:
        csv_2s: BERT-PSIE提取的 single-mentioned 的 CSV 文件路径
        csv_3: BERT-PSIE提取的 multiple-mentioned 的 CSV 文件路径
        xlsx_path: 输出的Excel文件路径: "FINAL_{CODE}_{dtime}.xlsx"
    """
    # 读取并合并CSV文件
    df1 = pd.read_csv(csv_2s)
    df2 = pd.read_csv(csv_3)
    df1.columns = df2.columns
    df = pd.concat([df1, df2], ignore_index=True)

    df = df.iloc[:, 1:]  # 去除第一列
    df.columns = ['material', 'value', 'sentence', 'doi']  # rename
    df = df[['sentence', 'doi', 'material', 'value']]  # reorder
    with pd.ExcelWriter(xlsx_path, mode="w", engine="openpyxl") as writer:
        df.to_excel(writer, sheet_name="1-raw", index=False)
    with pd.ExcelWriter(comparison_xlsx, mode="a", if_sheet_exists="replace", engine="openpyxl") as writer:
        df.to_excel(writer, sheet_name=f"{code}_raw", index=False)

def postprocess_psie(csv_2s, csv_3, xlsx_path, comparison_xlsx, code):
    """提取流程的主函数
    
    Args:
        csv_2s: BERT-PSIE提取的 single-mentioned 的 CSV 文件路径
        csv_3: BERT-PSIE提取的 multiple-mentioned 的 CSV 文件路径
        xlsx_path: 结果Excel - "FINAL_{CODE}_{dtime}.xlsx"
        comparison_xlsx: 标准答案文件路径
        code: 提取方法代码
    """
    post_psie(csv_2s, csv_3, xlsx_path, comparison_xlsx, code)
    clean_and_normalize(xlsx_path, comparison_xlsx, code)
    compare(xlsx_path, comparison_xlsx, code)

OUTPUT_DIR = os.path.join(HOME, "output", "2-BERT-PSIE")
OUTPUT_CSV_2_S = os.path.join(OUTPUT_DIR, "2-test_extraction_single_mentions.csv")
OUTPUT_CSV_3 = os.path.join(OUTPUT_DIR, "3-relations_extraction.csv")

COMPARISON_XLSX = os.path.join(HOME, "comparison.xlsx")
CODE = "PSIE"
XLSX_PATH = os.path.join(HOME, f"2_{CODE}_{dtime}.xlsx")

postprocess_psie(OUTPUT_CSV_2_S, OUTPUT_CSV_3, XLSX_PATH, COMPARISON_XLSX, CODE)


## 3-CE
kernel: lc

In [ ]:
"""extract"""
import re
import os
import glob
import pandas as pd
from tqdm import tqdm
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
import logging
from datetime import datetime
dtime = datetime.now().strftime("%m%d-%H%M")
import time
import re


def inference(MODEL, CODE, PROPERTY, TXT_DIR, OUTPUT_DIR, logger):
    # 第一阶段：过滤句子
    def get_positive_sentences(model_name):
        MODEL_NAME = model_name.replace(":", "-")
        POSITIVE_CSV = os.path.join(OUTPUT_DIR, f"1_positive_sentences_{MODEL_NAME}_{CODE}_{dtime}.csv")

        classif_q = f'Is the following sentence related to "{PROPERTY}"? Answer only "Yes" or "No" without any explanation:'

        # 初始化或加载已有结果（读取已保存的CSV文件，如果不存在则创建一个空的DataFrame）
        try:
            df_positive = pd.read_csv(POSITIVE_CSV)
            processed_dois = set(df_positive["doi"].unique())
        except FileNotFoundError:
            df_positive = pd.DataFrame(
                columns=[
                    "original_index",
                    "positive_sentences",
                    "integrated_sentences",
                    "doi",
                ]
            )
            processed_dois = set()

        # 初始化模型
        llm = ChatOllama(model=model_name, temperature=0)
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "You are an expert extraction algorithm specialized in materials science.",
                ),
                ("human", "{question}\n{text}\n"),
            ]
        )
        chain = prompt | llm

        # 处理
        txt_files = glob.glob(os.path.join(TXT_DIR, "*.txt"))
        logger.info(
            f"Found {len(txt_files)} text files for processing with {model_name}"
        )

        for txt_path in tqdm(txt_files, desc=f"Processing {model_name}"):
            doi = os.path.basename(txt_path).replace(".txt", "").replace("_", "/")
            if doi in processed_dois:
                continue

            # 1. [TXT] -> [句子列表]
            with open(txt_path, "r", encoding="utf-8") as f:
                sentences = [line.strip() for line in f]

            # 2. 判断
            results = []
            for idx, sentence in enumerate(sentences):
                try:
                    answer = chain.invoke(
                        {"question": classif_q, "text": sentence}
                    ).content
                    answer = re.sub(r"[^\w\s]", "", answer).strip().lower()
                    results.append((idx, sentence, 1 if answer == "yes" else 0))
                except Exception as e:
                    logger.error(f"Error processing sentence {idx} in {doi}: {str(e)}")
                    results.append((idx, sentence, 0))

            # 3. 生成集成句子（Sentence_Before + Sentence_Itself）
            positive_data = []
            for idx, (sentence_idx, sentence, label) in enumerate(results):
                if label == 1:
                    integrated = (
                        f"{results[idx-1][1]} {sentence}" if idx > 0 else sentence
                    )
                    positive_data.append(
                        {
                            "original_index": f"{sentence_idx}/{len(sentences)}",
                            "positive_sentences": sentence,
                            "integrated_sentences": integrated,
                            "doi": doi,
                        }
                    )

            # 定期保存
            if positive_data:
                df_positive = pd.concat([df_positive, pd.DataFrame(positive_data)])
                df_positive.to_csv(POSITIVE_CSV, index=False)
                logger.info(f"Saved {len(positive_data)} positive sentences from {doi}")

        return POSITIVE_CSV

    # 第二阶段：数据提取
    def extract_data(model_name, csv_path):
        TEMPERATURE = 0
        CONTEXT = 4096  # Default: 2048
        MODEL_NAME = model_name.replace(":", "-")
        os.makedirs(OUTPUT_DIR, exist_ok=True)

        # 初始化输出文件
        EXTRACTED_CSV = os.path.join(
            OUTPUT_DIR, f"2_extracted_{MODEL_NAME}_{CODE}_{dtime}.csv"
        )
        BINCLAS_CSV = os.path.join(OUTPUT_DIR, f"2_binclas_{MODEL_NAME}_{CODE}_{dtime}.csv")
        DIALOGUE_CSV = os.path.join(OUTPUT_DIR, f"2_dialogues_{MODEL_NAME}_{CODE}_{dtime}.csv")

        # 创建带标题的空CSV文件
        pd.DataFrame(columns=[
            "passage", "sentence", "doi", "material", "value", "unit",
            "material_valid", "value_valid", "unit_valid"
        ]).to_csv(EXTRACTED_CSV, index=False)

        # PROMPTS
        classif_q = f'Answer only "Yes" or "No" without any explanation. Based on the following text, is there a value of **{PROPERTY}** mentioned in it?\n\n'  # wk changed
        ifmulti_q = f'Answer "Yes" or "No" only. Does the following text mention more than one value of **{PROPERTY}**?\n\n'  # wk changed
        single_q = [
            f'Give the number only without units, do not use a full sentence. If the value is not present in the text, type "None". What is the value of the **{PROPERTY}** in the following text?\n\n',
            f'Give the unit only, do not use a full sentence. If the unit is not present in the text, type "None". What is the unit of the **{PROPERTY}** in the following text?\n\n',
            f'Give the name of the material only, do not use a full sentence. If the name of the material is not present in the text, type "None". What is the material for which the **{PROPERTY}** is given in the following text?\n\n',
        ]
        singlefollowup_q = [
            [
                'There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',
                f" the value of the **{PROPERTY}** for the material in the following text?\n\n",
            ],
            [
                'There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',
                f" the unit of the value of **{PROPERTY}** in the following text?\n\n",
            ],
            [
                'There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',
                f" the material for which the value of **{PROPERTY}** is given in the following text? Make sure it is a real material.\n\n",
            ],
        ]

        tab_q = f'Use only data present in the text. If data is not present in the text, type "None". Summarize the values of **{PROPERTY}** in the following text in a form of a table consisting of: Material, Value, Unit. Ensure that the "Value" and "Unit" are separated into different columns.\n\n'  # wk changed
        tabfollowup_q = [
            [
                'There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',
                " the ",
                f" material for which the value of **{PROPERTY}** is given in the following text? Make sure it is a real material.\n\n",
            ],
            [
                'There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',
                f" the value of the **{PROPERTY}** for the ",
                " material in the following text?\n\n",
            ],
            [
                'There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',
                " the unit of the ",
                f" value of **{PROPERTY}** in the following text?\n\n",
            ],
        ]

        it = [
            "first",
            "second",
            "third",
            "fourth",
            "fifth",
            "sixth",
            "seventh",
            "eighth",
            "ninth",
            "tenth",
            "eleventh",
            "twelfth",
            "thirteenth",
            "fourteenth",
            "fifteenth",
            "sixteenth",
            "seventeenth",
            "eighteenth",
            "nineteenth",
            "twentieth",
        ]
        col = ["Material", "Value", "Unit"]

        single_cols = ["value", "unit", "material"]

        # 初始化模型
        llm = ChatOllama(model=model_name, temperature=TEMPERATURE, num_ctx=CONTEXT)
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "You are an expert extraction algorithm specialized in materials science.",
                ),
                ("placeholder", "{conversation}"),
            ]
        )
        chain = prompt | llm

        try:
            df = pd.read_csv(csv_path)
            ntot = len(df)
            logger.info(f"Starting data extraction for {len(df)} entries")
        except Exception as e:
            logger.error(f"Failed to read CSV: {str(e)}")
            return

        with tqdm(total=ntot, desc=f"Extracting {MODEL_NAME}") as pbar:
            for i in range(ntot):
                try:
                    binary_classif = []
                    sss = []
                    sss.append(("human", classif_q + df["positive_sentences"][i]))
                    ans = chain.invoke({"conversation": sss}).content
                    sss.append(("ai", ans))
                    if "yes" in ans.strip().lower():  # wk: positive
                        binary_classif.append(1)
                        result = {}
                        passage = df["integrated_sentences"][i]
                        sentence = df["positive_sentences"][i]
                        sss.append(("human", ifmulti_q + passage))
                        ans = chain.invoke({"conversation": sss}).content
                        sss.append(("ai", ans))
                        if "no" in ans.lower():  # wk: positive; single data
                            result["passage"] = [passage]
                            result["sentence"] = [sentence]
                            result["doi"] = [df["doi"][i]]
                            result["material"] = []
                            result["value"] = []
                            result["unit"] = []
                            result["material_valid"] = []
                            result["value_valid"] = []
                            result["unit_valid"] = []
                            for j in range(
                                len(single_q)
                            ):  # wk: iterate 'single data' questions
                                sss.append(("human", single_q[j] + passage))
                                ans = chain.invoke({"conversation": sss}).content
                                sss.append(("ai", ans))
                                result[single_cols[j]].append(ans)
                                if "none" in ans.lower():
                                    result[single_cols[j] + "_valid"].append(0)
                                else:
                                    result[single_cols[j] + "_valid"].append(1)
                        elif "yes" in ans.lower():  # wk: positive; multiple data
                            sss.append(("human", tab_q + passage))
                            tab = chain.invoke({"conversation": sss}).content
                            sss.append(("ai", tab))
                            sst = copy(sss)
                            tab = [
                                split("[,|]", row) for row in tab.strip().split("\n")
                            ]
                            tab = [
                                [item.strip() for item in row if len(item.strip()) > 0]
                                for row in tab
                                if len(row) >= 3
                            ]
                            if len(tab) <= 0:
                                tab.append(["Material", "Value", "Unit"])
                            if len(tab) <= 1:
                                tab.append(["None", "None", "None"])
                            else:
                                tab.pop(1)
                            head = tab.pop(0)
                            tab = pd.DataFrame(
                                tab, columns=head
                            )  # wk: change (markdown?) format table to DataFrame
                            result["passage"] = []
                            result["sentence"] = []
                            result["doi"] = []
                            result["material"] = []
                            result["value"] = []
                            result["unit"] = []
                            result["material_valid"] = []
                            result["value_valid"] = []
                            result["unit_valid"] = []
                            for k in range(len(tab)):
                                sst.append(
                                    (
                                        "tab",
                                        tab[col[0]][k]
                                        + ","
                                        + tab[col[1]][k]
                                        + ","
                                        + tab[col[2]][k],
                                    )
                                )
                                result["passage"].append(passage)
                                result["sentence"].append(sentence)
                                result["doi"].append(df["doi"][i])
                                multi_valid = True
                                for l in range(3):
                                    ss = (
                                        tabfollowup_q[l][0]
                                        + str(tab[col[l]][k])
                                        + tabfollowup_q[l][1]
                                        + it[k]
                                        + tabfollowup_q[l][2]
                                        + passage
                                    )
                                    result[col[l].lower()].append(tab[col[l]][k])
                                    if "none" in tab[col[l]][k].lower():
                                        result[col[l].lower() + "_valid"].append(0)
                                        multi_valid = False
                                    elif multi_valid:
                                        sss.append(("human", ss))
                                        sst.append(("human", ss))
                                        ans = chain.invoke(
                                            {"conversation": sss}
                                        ).content
                                        sss.append(("ai", ans))
                                        sst.append(("ai", ans))
                                        if "no" in ans.lower():
                                            result[col[l].lower() + "_valid"].append(0)
                                            multi_valid = False
                                        else:
                                            result[col[l].lower() + "_valid"].append(1)
                                    else:
                                        result[col[l].lower() + "_valid"].append(1)
                        try:  # wk: [OUTPUT1]
                            pd.DataFrame(result).to_csv(
                                EXTRACTED_CSV, mode="a", index=False, header=False
                            )
                        except Exception as e:
                            print("Appending extracted gone wrong: ", i, "  ", e)
                            print("Appending extracted gone wrong: ", result, "  ", e)
                            print("Appending extracted gone wrong: ", tab, "  ", e)
                    else:  # wk: negative
                        binary_classif.append(0)
                    pd.DataFrame(binary_classif).to_csv(
                        BINCLAS_CSV, mode="a", index=False, header=False
                    )  # wk: [OUTPUT2] positive or negative
                    try:  # wk: [OUTPUT3] save all the conversations with GPT (save sst, if sst exsist)
                        pd.DataFrame(sst).to_csv(
                            DIALOGUE_CSV, mode="a", index=False, header=False
                        )
                        del sst
                    except:
                        pd.DataFrame(sss).to_csv(
                            DIALOGUE_CSV, mode="a", index=False, header=False
                        )
                    pbar.update(1)
                except Exception as e:
                    logger.error(f"Error processing row {i}: {str(e)}")
                    print(f"Ignoring {i+1}/{ntot} ({round(i/ntot*100,1)} %)")
                    continue

        logger.info(f"Completed data extraction for {model_name}")
        return EXTRACTED_CSV

    # 主执行流程
    stage1_output = get_positive_sentences(MODEL)
    EXTRACTED_CSV = extract_data(MODEL, stage1_output)
    return EXTRACTED_CSV

def process_chat(MODELS, PROPERTY, TXT_DIR, OUTPUT_DIR):
    # 初始化日志配置
    start_time = time.time()
    log_file = os.path.join(OUTPUT_DIR, f"processing_{dtime}.log")

    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")

    # 文件日志处理器
    file_handler = logging.FileHandler(log_file)
    file_handler.setFormatter(formatter)

    # 控制台日志处理器
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    logger.info("Initializing inference pipeline...")
    logger.info(f"Property: {PROPERTY}")
    logger.info(f"Input TXT directory: {TXT_DIR}")
    logger.info(f"Output directory: {OUTPUT_DIR}")

    for MODEL, CODE in MODELS.items():
        logger.info(f"\n{'='*40}")
        logger.info(f"Processing model: {MODEL}")
        inference(MODEL, CODE, PROPERTY, TXT_DIR, OUTPUT_DIR, logger)
        logger.info(f"Completed processing for {MODEL}")

    logger.info(f"\n{'='*40}")
    logger.info(f"Total processing time: {time.time()-start_time:.2f} seconds")
    logger.info(f"Output directory: {OUTPUT_DIR}")
    logger.info("Processing complete!")


# 执行
TXT_DIR = os.path.join(HOME, "TXT(fromPDF_processed)")
OUTPUT_DIR = os.path.join(HOME, "output", "3-ChatExtract")
os.makedirs(OUTPUT_DIR, exist_ok=True)

MODELS = {
    "llama2:13b": "CE_1",
    "llama3.1:70b": "CE_2",
    "qwen2.5:14b": "CE_3",
}

process_chat(
    MODELS=MODELS,
    PROPERTY="band gap",
    TXT_DIR=TXT_DIR,
    OUTPUT_DIR=OUTPUT_DIR,
    
)

In [ ]:
"""postprocessing"""
import os
import pandas as pd
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from my_post import clean_and_normalize, compare

def post_ce(extracted_csv, xlsx_path, comparison_xlsx, code):
    """ChatExtract提取的原始数据转换为标准格式
    
    Args:
        extracted_csv: "extracted_{model}_{dtime}.csv"
        xlsx_path: 输出的Excel文件路径: "FINAL_{CODE}_{dtime}.xlsx"
    """
    # 读取CSV文件
    df = pd.read_csv(extracted_csv)
    # 筛选material_valid、value_valid和unit_valid均为1的行
    df = df[(df["material_valid"] == 1) &
            (df["value_valid"] == 1) &
            (df["unit_valid"] == 1)
            ]
    # 去除value列中的"meV"或"eV"
    df["value"] = df["value"].apply(lambda x: x.replace("meV", "").replace("eV", "").strip() if pd.notna(x) else x)
    # 删去不要的列并重新排序
    columns_to_keep = ["doi", "material", "value", "unit", "passage", "sentence"]
    df = df[columns_to_keep]
    # 清洗非法控制字符的函数
    def clean_illegal_chars(text):
        if isinstance(text, str):
            # 移除控制字符（保留制表符\t、换行符\n、回车符\r）
            return re.sub(r"[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F-\x9F]", "", text)
        else:
            return text
    df = df.map(clean_illegal_chars)  # 应用清洗函数到整个DataFrame

    with pd.ExcelWriter(xlsx_path, mode="w", engine="openpyxl") as writer:
        df.to_excel(writer, sheet_name="1-raw", index=False)
    with pd.ExcelWriter(comparison_xlsx, mode="a", if_sheet_exists="replace", engine="openpyxl") as writer:
        df.to_excel(writer, sheet_name=f"{code}_raw", index=False)

def postprocess_ce(extracted_csv, xlsx_path, comparison_xlsx, code):
    """CDE提取流程的主函数
    
    Args:
        extracted_csv: "extracted_{model}_{dtime}.csv"
        xlsx_path: 结果Excel - "FINAL_{CODE}_{dtime}.xlsx"
        comparison_xlsx: 标准答案文件路径
        code: 提取方法代码
    """
    post_ce(extracted_csv, xlsx_path, comparison_xlsx, code)
    clean_and_normalize(xlsx_path, comparison_xlsx, code)
    compare(xlsx_path, comparison_xlsx, code)

COMPARISON_XLSX = os.path.join(HOME, "comparison.xlsx")

DATA_DIR = OUTPUT_DIR
CSVS = [f'2_extracted_llama2-13b_CE_1_{dtime}.csv',
        f'2_extracted_llama3.1-70b_CE_2_{dtime}.csv',
        f'2_extracted_qwen2.5-14b_CE_3_{dtime}.csv']
CODES = ["CE_1",  # llama2:13b
         "CE_2",  # llama3.1:70b
         "CE_3",  # qwen2.5:14b
         ]
for csv, code in zip(CSVS, CODES):
    extracted_csv = os.path.join(DATA_DIR, csv)
    xlsx_path = os.path.join(HOME, f"3_{code}_{dtime}.xlsx")
    postprocess_ce(extracted_csv, xlsx_path, COMPARISON_XLSX, code)


## 4-LC
kernel: lc

In [ ]:
"""extract"""
import os
import glob
import pandas as pd
from tqdm import tqdm
import gc
import time
from datetime import datetime
dtime = datetime.now().strftime("%m%d-%H%M")
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from chromadb import Client
from chromadb.config import Settings

def process_single_pdf(pdf_file, embeddings, llm, text_splitter, prompt, question, client):
    """处理单个PDF文件并提取信息
    
    Args:
        pdf_file: PDF文件路径
        embeddings: 嵌入模型
        llm: 语言模型
        text_splitter: 文本分割器
        prompt: 提示模板
        question: 查询问题
        client: ChromaDB客户端
    
    Returns:
        tuple: (doi, output, status)
    """
    doi = os.path.basename(pdf_file).replace('.pdf', '').replace('_', '/')
    try:
        collection_name = f"collection_{doi.replace('/', '_')}"  # 用DOI生成唯一名称
        try:  # 强制删除旧集合
            client.delete_collection(collection_name)
        except:
            pass
        # 加载和分割文档
        loader = PyMuPDFLoader(file_path=pdf_file)
        docs = loader.load()
        chunks = text_splitter.split_documents(docs)
        # 创建向量数据库
        vector_db = Chroma.from_documents(
            documents=chunks,
            embedding=embeddings,
            collection_name=collection_name,
            client=client
        )
        # 构建和执行RAG链
        def format_docs(docs):
            return "\n\n".join(doc.page_content for doc in docs)
        retriever = vector_db.as_retriever(search_kwargs={"k": 5})
        rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt
            | llm
        )
        output = rag_chain.invoke(question).content
        status = 1 if output.strip().startswith("|") else 0

        # 清理资源
        del loader, docs, chunks, vector_db, retriever, rag_chain
        gc.collect()
        
        return doi, output, status

    except Exception as e:
        return doi, f"处理错误: {str(e)}", -1

def process_pdfs_batch(codes, embed_stream, infer_stream, output_dir, pdf_dir, template, question):
    """批量处理PDF文件
    
    Args:
        codes: 处理代码列表
        embed_stream: 模型配对列表（每个嵌入模型配对所有推理模型）
        infer_stream: 模型配对列表（每个嵌入模型配对所有推理模型）
        output_dir: 输出目录
        pdf_dir: PDF文件目录
        template: 提示模板
        question: 查询问题
    """
    # 创建全局ChromaDB客户端
    client_settings = Settings(persist_directory="")
    global_client = Client(client_settings)  # 使用内存数据库，避免文件残留；关键：禁用持久化
    
    total_start_time = time.time()  # log
    for code, embed_model, infer_model in zip(codes, embed_stream, infer_stream):
        output_csv = os.path.join(output_dir, f"output_{code}_{dtime}.csv")
        # 初始化模型和工具
        embeddings = OllamaEmbeddings(model=embed_model)
        llm = ChatOllama(model=infer_model, temperature=0, num_predict=80)
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        prompt = PromptTemplate.from_template(template)

        # 处理每个PDF文件
        pdf_files = sorted(glob.glob(os.path.join(pdf_dir, "*.pdf")))
        start_time = time.time()  # log
        for pdf_file in tqdm(pdf_files, desc=f"处理PDF文件 - {code}"):
            doi, output, status = process_single_pdf(
                pdf_file, embeddings, llm, text_splitter, prompt, question, global_client
            )
            # 保存结果
            df_new = pd.DataFrame([{'doi': doi, 'output': output, 'status': status}])
            if os.path.exists(output_csv):
                df_new.to_csv(output_csv, mode='a', header=False, index=False)
            else:
                df_new.to_csv(output_csv, index=False)
        end_time = time.time()  # 结束计时
        batch_time = end_time - start_time
        # 写入日志文件
        with open(log_file, 'a') as lf:
            log_entry = (
                f"Code: {code} | Embed Model: {embed_model} | Infer Model: {infer_model}\n"
                f"Start: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))} | "
                f"End: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end_time))} | "
                f"Duration: {batch_time:.2f} seconds | Status: {'Success' if status == 1 else 'Error'}\n"
                f"{'='*40}\n"
            )
            lf.write(log_entry)
    total_end_time = time.time()  # 记录总结束时间
    total_time = total_end_time - total_start_time
    with open(log_file, 'a') as lf:
        log_entry = (
            f"Total Start: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(total_start_time))}\n"
            f"Total End: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(total_end_time))}\n"
            f"Total Duration: {total_time:.2f} seconds\n"
            f"{'='*40}\n"
        )
        lf.write(log_entry)

TEMPLATE = """
You are an expert information extraction algorithm.
Extract all the band gap values in the CONTEXT given blow.
Output the band gap values in the form of a markdown table, including: Material (name of the material), Value (band gap value), Unit (unit of value).
Do not explain, only output the table in markdown format.
The output is strictly in the following format.
| Material | Value | Unit |
|----------|-------|------|
| ... | ... | eV |
| ... | ... | meV |
If no band gap values mentioned in the article, the following table is acceptable:
| Material | Value | Unit |
|----------|-------|------|
| None | None | None |
---
CONTEXT: {context}
---
QUESTION: {question}
Answer in markdown table:
"""
QUESTION = "What are the materials' name and their band gap values?"

TXT_DIR = os.path.join(HOME, "TXT(fromPDF_processed)")
PDF_DIR = os.path.join(HOME, "PDF")
OUTPUT_DIR = os.path.join(HOME, "output", "4-LangChain")
os.makedirs(OUTPUT_DIR, exist_ok=True)
log_file = os.path.join(OUTPUT_DIR, f"log_{dtime}.log")
EMBEDDING_MODELS = [
    "nomic-embed-text",
    "bge-m3",
    ]
INFERENCE_MODELS = [
    "llama2:13b",
    "llama3.1:70b",
    "qwen2.5:14b",
    ]
CODES = [f"LC_{i+1}{j+1}" 
        for i in range(len(EMBEDDING_MODELS)) 
        for j in range(len(INFERENCE_MODELS))]
embed_stream = [model for model in EMBEDDING_MODELS for _ in INFERENCE_MODELS]
infer_stream = INFERENCE_MODELS * len(EMBEDDING_MODELS)

# 执行批处理
process_pdfs_batch(
    CODES,
    embed_stream,
    infer_stream,
    OUTPUT_DIR,
    PDF_DIR,
    TEMPLATE,
    QUESTION
)

In [ ]:
"""postprocessing"""
import os
import pandas as pd
import re
from datetime import datetime
from io import StringIO
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from my_post import clean_and_normalize, compare

def post_lc(extracted_csv, xlsx_path, comparison_xlsx, code):
    """将LangChain提取的原始数据转换为标准格式
    
    将CSV格式的原始数据转换为标准的Excel格式，包含doi、material和value三列。
    处理过程包括:
    1. 读取并去重原始数据
    2. 解析每行中的markdown表格
    3. 标准化表格格式并合并结果
    
    Args:
        extracted_csv (str): 输入的CSV文件路径，格式为"output_{embed}_{infer}_{dtime}.csv"
        xlsx_path (str): 输出的Excel文件路径，格式为"FINAL_{embed}_{infer}_{dtime}.xlsx"
    """
    # 读取CSV并去重
    raw_df = pd.read_csv(extracted_csv)
    raw_df = raw_df.drop_duplicates(subset=["output"], keep="first")
    # 只保留status为1的行
    raw_df = raw_df[raw_df['status'] == 1]
    # 初始化结果DataFrame
    result_df = pd.DataFrame(columns=["doi", "material", "value", "unit"])
    
    def parse_markdown_table(output, doi):
        """解析单个markdown表格
        
        Args:
            output (str): markdown格式的表格字符串
            doi (str): 对应的DOI
            
        Returns:
            pd.DataFrame: 解析后的表格，若解析失败返回None
        """
        try:
            # 解析markdown表格
            table = pd.read_csv(StringIO(output), sep="|", skipinitialspace=True)
            table = table.dropna(axis=1, how="all")  # 删除空列
            table = table.iloc[1:, :]  # 删除表头行
            
            # 验证并格式化表格
            if len(table.columns) == 3:
                table.columns = ["material", "value", "unit"]
                table["material"] = table["material"].str.strip()
                table["doi"] = doi
                return table[["doi", "material", "value", "unit"]]
            return None
        except Exception:
            print(f"无法处理DOI为{doi}的表格")
            return None
    
    # 处理每一行数据
    parsed_tables = []
    for _, row in raw_df.iterrows():
        table = parse_markdown_table(row["output"], row["doi"])
        if table is not None:
            parsed_tables.append(table)
    # 合并、保存所有解析结果
    if parsed_tables:
        result_df = pd.concat(parsed_tables, ignore_index=True)
    
    """简单处理"""
    # 去除value列中的"meV"或"eV"
    result_df["value"] = result_df["value"].apply(lambda x: x.replace("meV", "").replace("eV", "").strip() if pd.notna(x) else x)
    # 清洗非法控制字符的函数
    def clean_illegal_chars(text):
        if isinstance(text, str):
            # 移除控制字符（保留制表符\t、换行符\n、回车符\r）
            return re.sub(r"[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F-\x9F]", "", text)
        else:
            return text
    result_df = result_df.map(clean_illegal_chars)  # 应用清洗函数到整个DataFrame
    
    with pd.ExcelWriter(xlsx_path, mode="w", engine="openpyxl") as writer:
        result_df.to_excel(writer, sheet_name="0-raw", index=False)
    
    # 删去空值
    result_df = result_df[~result_df["value"].str.lower().str.strip().isin(["none", "nan"]) & result_df["value"].notna()]
    with pd.ExcelWriter(xlsx_path, mode="w", engine="openpyxl") as writer:
        result_df.to_excel(writer, sheet_name="1-raw", index=False)
    with pd.ExcelWriter(comparison_xlsx, mode="a", if_sheet_exists="replace", engine="openpyxl") as writer:
        result_df.to_excel(writer, sheet_name=f"{code}_raw", index=False)

def postprocess_lc(extracted_csv, xlsx_path, comparison_xlsx, code):
    """CDE提取流程的主函数
    
    Args:
        extracted_csv: "extracted_{model}_{dtime}.csv"
        xlsx_path: 结果Excel - "FINAL_{CODE}_{dtime}.xlsx"
        comparison_xlsx: 标准答案文件路径
        code: 提取方法代码
    """
    post_lc(extracted_csv, xlsx_path, comparison_xlsx, code)
    clean_and_normalize(xlsx_path, comparison_xlsx, code)
    compare(xlsx_path, comparison_xlsx, code)

COMPARISON_XLSX = os.path.join(HOME, "comparison.xlsx")

DATA_DIR = OUTPUT_DIR
csv_files = glob.glob(os.path.join(DATA_DIR, "output_LC_*.csv"))
csv_files.sort()
# print(csv_files)
codes = [f"{os.path.basename(file).split('_')[1]}_{os.path.basename(file).split('_')[2]}" for file in csv_files]

for extracted_csv, code in zip(csv_files, codes):
    print(extracted_csv, code)
    xlsx_path = os.path.join(HOME, f"4_{code}_{dtime}.xlsx")
    postprocess_lc(extracted_csv, xlsx_path, COMPARISON_XLSX, code)


## 5-Kimi
kernel: lc

In [ ]:
"""postprocessing"""
import os
import pandas as pd
import re
from datetime import datetime
from io import StringIO
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from my_post import clean_and_normalize, compare

def post_kimi(extracted_xlsx, xlsx_path, comparison_xlsx, code):
    df = pd.read_excel(extracted_xlsx, sheet_name="1-Markdown")
    result_df = pd.DataFrame(columns=["doi", "material", "value", "sentence"])
    # 遍历'output'列
    for index, row in df.iterrows():
        doi = row["doi"].replace('_', '/')
        output = row["output"]
        # 手动处理每行，避免内容中的竖线被误分割
        lines = [line.strip() for line in output.split('\n') if line.strip()]
        # 提取表头
        headers = [h.strip() for h in lines[0].strip('| ').split('|')]
        headers = [h for h in headers if h]  # 去除空字符串
        # 处理数据行（跳过第二行的分隔线）
        data = []
        for line in lines[2:]:
            # 去除行首尾的管道符并按列分割
            stripped_line = line.strip('| ')
            parts = [p.strip() for p in stripped_line.split('|')]
            # 确保每行有三列（Material, Value, Sentence）
            if len(parts) == 3:
                data.append(parts)
            else:
                # 处理异常行（例如内容中有竖线但被错误分割）
                # 这里简单示例，实际需根据数据调整
                material = parts[0].strip()
                value = parts[1].strip()
                sentence = '|'.join(parts[2:]).strip()  # 合并多余部分
                data.append([material, value, sentence])
        # 创建DataFrame
        table = pd.DataFrame(data, columns=headers)
        # 删除空白列和标题行
        table = table.dropna(axis=1, how="all").iloc[1:]
        # 确保表格为三列
        if len(table.columns) == 3:
            # 重命名列
            table.columns = ["material", "value", "sentence"]
            # 添加doi列
            table["doi"] = doi
            # 调整列顺序
            table = table[["doi", "material", "value", "sentence"]]
            # 将处理后的表格添加到结果DataFrame
            result_df = pd.concat([result_df, table], ignore_index=True)
    # 保存结果
    with pd.ExcelWriter(xlsx_path, mode="w", engine="openpyxl") as writer:
        result_df.to_excel(writer, sheet_name="1-raw", index=False)
    with pd.ExcelWriter(comparison_xlsx, mode="a", if_sheet_exists="replace", engine="openpyxl") as writer:
        result_df.to_excel(writer, sheet_name=f"{code}_raw", index=False)

def postprocess_kimi(extracted_xlsx, xlsx_path, comparison_xlsx, code):
    """CDE提取流程的主函数
    
    Args:
        xlsx_path: 结果Excel - "FINAL_{CODE}_{dtime}.xlsx"
        comparison_xlsx: 标准答案文件路径
        code: 提取方法代码
    """
    post_kimi(extracted_xlsx, xlsx_path, comparison_xlsx, code)
    clean_and_normalize(xlsx_path, comparison_xlsx, code)
    compare(xlsx_path, comparison_xlsx, code)

dtime = datetime.now().strftime("%m%d-%H%M")
COMPARISON_XLSX = os.path.join(HOME, "comparison.xlsx")
CODE = "Kimi"
KIMI_OUT = os.path.join(HOME, "output", "5-Kimi", 'kimi.20250217221417285.xlsx')
xlsx_path = os.path.join(HOME, f"5_{CODE}_{dtime}.xlsx")
postprocess_kimi(KIMI_OUT, xlsx_path, COMPARISON_XLSX, CODE)


# 3 Organization

In [12]:
import pandas as pd
import os

COMPARISON_XLSX = os.path.join(HOME, "comparison.xlsx")

df = pd.read_excel(COMPARISON_XLSX, sheet_name="summary")
# 删去manual之外的内容
df.dropna(subset=['material'], inplace=True)
df.drop(columns=["other_mat"], inplace=True)

# 读取error之后的所有列的内容
methods = df.columns[df.columns.str.contains('_error')].str.replace('_error', '').unique()  # 获取所有方法名
# 删去所有的error列
df = df.loc[:, ~df.columns.str.contains('_error')]
for method in methods:
    method_col = f"{method}"
    # 如果f“{method}”列某一行不为空，则替换为1
    df[method_col] = df[method_col].apply(lambda x: 1 if pd.notna(x) else x)

# 保存处理后的表格在同一个xlsx文件下的新sheet“PRF”
with pd.ExcelWriter(COMPARISON_XLSX, mode="a", if_sheet_exists="replace", engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="PRF", index=False)
